## 1.1 Install Dependencies



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from keras.layers import Flatten, Dense
from keras.models import Model 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img,img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
from sklearn.model_selection import train_test_split

## 1.2 read data


In [2]:
import os
path='mask'
img_list=os.listdir(path)
len(img_list)
dic=['without_mask', 'with_mask']
data=[]
label=[]
for cat in dic:
    pic_list=os.path.join(path,cat)
    for img in  os.listdir(pic_list):
        image=os.path.join(pic_list,img)
     #   print(image)
        image=load_img(image, target_size=(224, 224))
        image=img_to_array(image)
        #print(image)
        image=preprocess_input(image)
        data.append(image)
        label.append(cat)       
    #print(pic_list)

C:\Users\hp\anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [3]:
lb = LabelBinarizer()
label = lb.fit_transform(label)
label = to_categorical(label)




In [4]:
data = np.array(data, dtype='float32')
label = np.array(label)
data.shape



(3833, 224, 224, 3)

In [5]:
label.shape

(3833, 2)

## 1.3 read data


In [ ]:

X_train, X_test, y_train,  y_test= train_test_split(data, label, test_size=0.2, shuffle=True ,random_state=1)



## 1.3 DataGenerator 


In [ ]:
augment = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)



# 1.4 Mask and Social distancing Detection using MobileNetV2














In [ ]:
from tensorflow.keras.applications import MobileNetV2
from keras.layers import Input,Dropout ,AveragePooling2D
mobileNet = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
headModel = mobileNet.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name='flatten')(headModel)
headModel = Dense(128, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation='softmax')(headModel)
model = Model(inputs=mobileNet.input, outputs=headModel)

In [ ]:
model.summary()

In [ ]:
lr = 1e-4
epochs = 20
bs = 32
from keras.optimizers import Adam
#freeze some layers
for layer in mobileNet.layers:
  layer.trainable = False

#optmizer = Adam(learning_rate=lr, decay=lr/epochs)
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(augment.flow(X_train, y_train, batch_size=bs),
                              steps_per_epoch=len(X_train)//bs, epochs=epochs,
                              validation_data=(X_test, y_test), shuffle=1)

In [ ]:
h=history.history

In [ ]:
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'] , c = "red")
plt.title("acc vs v-acc")
plt.plot(h['loss'])
plt.plot(h['val_loss'])
plt.legend(['loss','val_loss'])
plt.xlabel('num-of iter')
plt.title('loss vs loss-val')

plt.legend(['accuracy', 'val_accuracy','loss','val_loss'])
plt.show()

In [ ]:
plt??

In [ ]:

plt.show()

# save model 

In [ ]:
model.save('face_detection.h5')

In [ ]:
from keras.models import load_model

In [ ]:
new_model=load_model('face_detection.h5')

In [ ]:

predections =new_model.predict(X_test)
predections = np.argmax(predections, axis = 1)

In [ ]:

print(classification_report(y_test.argmax(axis=1), predections, target_names=lb.classes_))

In [ ]:
 mask_result = new_model.predict(X_test)